#Setup:

1. Use pynvml to confirm Colab allocated you a Tesla T4 GPU.
2. Install most recent Miniconda release compatible with Google Colab's Python install  (3.6.7)
3. Install RAPIDS libraries
4. Copy RAPIDS .so files into current working directory, a workaround for conda/colab interactions
5. Add the ngrok binary to expose Dask's status dashboard
6. Update env variables so Python can find and use RAPIDS artifacts
​
All of the above steps are automated in the next cell.
​
You should re-run this cell any time your instance re-starts.

In [0]:
"""install RAPIDS AI suite and set up environment
>> may take a few minutes, long output (output display removed)
"""
!wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
!bash rapids-colab.sh

import sys, os

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# Weighted Jaccard Similarity

In this notebook, we will use cuGraph to compute the Weighted Jaccard Similarity metric on our training dataset. 

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 07/29/2019


## Introduction
Weighted Jaccard is similar to the Jaccard Similarity but takes into account vertex weights placed.  


given:
The neighbors of a vertex, v, is defined as the set, U, of vertices connected by way of an edge to vertex v, or N(v) = {U} where v ∈V and ∀ u∈U ∃ edge(v,u)∈E.
and
wt(i) is the weight on vertex i
   
we can now define weight summing function as<br>
$WT(U) = \sum_{v \in U} {wt(v)}$

$WtJaccard(i, j) = \frac{WT(N(i) \cap N(j))}{WT(N(i) \cup N(j))}$


To compute the weighted Jaccard similarity between each pair of vertices connected by an edge in cuGraph use:<br>

**jaccard_w(input_graph, vect_weights_ptr)**

Input
* input_graph: A cugraph.Graph object
* vect_weights_ptr: An array of vertex weights

Returns: 
* __df__: cudf.DataFrame with three columns:
    * df['source']: The source vertex id.
    * df['destination']: The destination vertex id.
    * df['jaccard_coeff']: The weighted jaccard coefficient computed between the source and destination vertex.
        
<br><br>   


__Note:__ For this example we will be using PageRank as the edge weights.  Please review the PageRank notebook if you have any questions about running PageRank


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](https://raw.githubusercontent.com/rapidsai/notebooks/branch-0.8/cugraph/img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  

In [0]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

## Read the data using cuDF

In [4]:
# Save test file
!wget https://raw.githubusercontent.com/rapidsai/notebooks/branch-0.8/cugraph/data/karate-data.csv
datafile='karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

--2019-07-29 09:50:20--  https://raw.githubusercontent.com/rapidsai/notebooks/branch-0.8/cugraph/data/karate-data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 814 [text/plain]
Saving to: ‘karate-data.csv’

karate-data.csv     100%[===================>]     814  --.-KB/s    in 0s      

2019-07-29 09:50:20 (143 MB/s) - ‘karate-data.csv’ saved [814/814]



In [0]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

### Compute PageRank to use as the vertex weights

In [0]:
# Call Pagerank on the graph to get weights to use:
pr_df = cugraph.pagerank(G)

In [7]:
pr_df.head().to_pandas()

,vertex,pagerank
0,0,0.004392
1,1,0.096573
2,2,0.052645
3,3,0.056828
4,4,0.035703


### Now compute the Weighted Jaccard 

In [0]:
# Call weighted Jaccard using the Pagerank scores as weights:
df = cugraph.jaccard_w(G, pr_df['pagerank'])

Find the most similar pair of vertices

In [9]:
bestEdge = 0
for i in range(len(df)):
    if df['jaccard_coeff'][i] > df['jaccard_coeff'][bestEdge]:
        bestEdge = i

# display outcome 
print("Vertices " + str(df['source'][bestEdge]) + 
      " and " + str(df['destination'][bestEdge] ) + 
      " are most similar with score: " + str(df['jaccard_coeff'][bestEdge]))

Vertices 4 and 8 are most similar with score: 0.6644074


## It is that easy with cuGraph!

# Next Steps #

For an overview of how you can access and work with your own datasets in Colab, check out [this guide](https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92).

For more RAPIDS examples, check out our RAPIDS notebooks repos:
1. https://github.com/rapidsai/notebooks
2. https://github.com/rapidsai/notebooks-extended

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___